# Producer Kafka

In [3]:
import time
from kafka import KafkaProducer
import csv

kafka_url = 'kafka:9092'

#Démarrer le producteur Kafka
producer = KafkaProducer(bootstrap_servers=kafka_url, value_serializer=lambda v: str(v).encode('utf-8'))

## Envoi de message

In [5]:
topic = 'topic-example'

for i in range(10):
    message = f'Message {i}'
    producer.send(topic, value=message)
    print(f'Envoyé:{message}')
    time.sleep(1) # Pause pour simuler un flux
    
producer.flush()
producer.close()

Envoyé:Message 0
Envoyé:Message 1
Envoyé:Message 2
Envoyé:Message 3
Envoyé:Message 4
Envoyé:Message 5
Envoyé:Message 6
Envoyé:Message 7
Envoyé:Message 8
Envoyé:Message 9


## Consumer Kafka

In [6]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    topic,
    bootstrap_servers=[kafka_url],
    auto_offset_reset ='earliest',
    consumer_timeout_ms=1000,
    value_deserializer=lambda value: value.decode('utf-8'),
    
)

print("Liste des messages reçus:")
for message in consumer:
    print(f'Reçu: {message.value}')
    
consumer.close()

Liste des messages reçus:
Reçu: Message 0
Reçu: Message 1
Reçu: Message 2
Reçu: Message 3
Reçu: Message 4
Reçu: Message 5
Reçu: Message 6
Reçu: Message 7
Reçu: Message 8
Reçu: Message 9


In [ ]:
# Open and read the CSV file
with open('data/test.csv', mode='r') as file:
    reader = csv.reader(file)
    header = next(reader)  # skip header if needed

    for row in reader:
        message = ','.join(row)  # convert row to string
        producer.send(topic, value=message)
        print(f'Envoyé: {message}')
        time.sleep(1)  # simulate streaming

producer.flush()
producer.close()

In [ ]:
kafka_url = 'kafka:9092'

# Démarrer le producteur Kafka
producer = KafkaProducer(bootstrap_servers=kafka_url, value_serializer=lambda v: str(v).encode('utf-8'))
topic2 = 'topic-example-file'

# Open and read the CSV file
with open('data/film.csv','r') as file:
    reader = csv.reader(file)
    header = next(reader)  # skip header if needed

    for i, row in enumerate(reader):
        if i >= 25:  # limit to first 100 rows
            break
        message = ','.join(row)  # convert row to string
        producer.send(topic2, value=message)
        print(f'Envoyé: {message}')
        time.sleep(1)  # simulate streaming

producer.flush()
producer.close()

In [51]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    topic2,
    bootstrap_servers=[kafka_url],
    auto_offset_reset='earliest',
    consumer_timeout_ms=1000,
    value_deserializer=lambda value: value.decode('utf-8'),
)

print("Liste des messages reçus:")
for message in consumer:
    print(f'Reçu: {message.value}')

consumer.close()

Liste des messages reçus:
Reçu: 1,ACADEMY DINOSAUR,An Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,1,,6,0.99,86,20.99,PG,Deleted Scenes,Behind the Scenes,2006-02-15 05:03:42
Reçu: 2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,1,,3,4.99,48,12.99,G,Trailers,Deleted Scenes,2006-02-15 05:03:42
Reçu: 3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,1,,7,2.99,50,18.99,NC-17,Trailers,Deleted Scenes,2006-02-15 05:03:42
Reçu: 4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,1,,5,2.99,117,26.99,G,Commentaries,Behind the Scenes,2006-02-15 05:03:42
Reçu: 5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,1,,6,2.99,130,22.99,G,Deleted Scenes,2006-

## Administration de Kafka depuis python

In [36]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers=kafka_url,
    client_id='test_admin'
)

topics = admin_client.list_topics()

In [ ]:
new_topic_name= 'topic-from-python'
if new_topic_name not in topics:
    topic = NewTopic(name=new_topic_name, num_partitions=1, replication_factor=1)
    admin_client.create_topics(new_topics=[topic], validate_only=False)
    print(f'Topic"{new_topic_name}" existe déjà.')
else:
    print(f'Topic"{new_topic_name}"existe déjà.')
    
# Supprinmer le topic

admin_client.delete_topics([new_topic_name]) 

NameError: name 'new_topic' is not defined

In [37]:
new_topic_name= 'topic-example'
admin_client.delete_topics([new_topic_name]) 

DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='topic-example', error_code=0)])

In [46]:
from kafka.admin import ConfigResource, ConfigResourceType

resource = ConfigResource(
    ConfigResourceType.TOPIC,
    topic, 
    configs={"retention.ms": "0"} # 0 milli-secondes
    
)

admin_client.alter_configs([ressource])
print(f'Configuration de rétention du topic"{topic2}" modifiée pour supprimer les messages immédiatement.')

# Remettre la configuration par défaut (7 jours)
resource = ConfigResource(
    ConfigResourceType.TOPIC,
    topics,
    configs={ "retention.ms": str(7 * 24 * 60 * 60 * 1000)} # 7 jours
)

admin_client.alter_configs([resource])
print(f'Configuration de rétention du topic "{topic2}"remise à 7 jours')

Configuration de rétention du topic"topic-example" modifiée pour supprimer les messages immédiatement.
Configuration de rétention du topic "topic-example"remise à 7 jours
